# Examples of WaporAPI module functions

## 0. Change working directory to "wapor/src"

In [1]:
import os
import glob

p_from = os.getcwd()
print('p_from')
for file in os.listdir(p_from):
    print('  ', file)

p_root = os.path.join(os.getcwd(), '../')
print('p_root')
for file in os.listdir(p_root):
    print('  ', file)

p_wapor = os.path.join(p_root, 'src')
p_tests = os.path.join(p_root, 'tests')

os.chdir(p_wapor)
print('From "{f}" to "{t}"'.format(f=p_from, t=p_wapor))
for file in os.listdir(os.getcwd()):
    print('  ', file)

p_from
   .ipynb_checkpoints
   test_bigtiff.ipynb
   test_gdal_driver.ipynb
   test_wapor.ipynb
p_root
   .coverage
   .coveragerc
   .eggs
   .git
   .gitignore
   .isort.cfg
   .pre-commit-config.yaml
   .pytest_cache
   .readthedocs.yml
   .travis.yml
   .vscode
   AUTHORS.rst
   build
   CHANGELOG.rst
   config.yml
   CONTRIBUTING.rst
   Docker
   Dockerfile
   docs
   examples
   LICENSE.rst
   notebooks
   PyCharm.jar
   README.md
   README.rst
   requirements-dev.txt
   requirements.txt
   setup.cfg
   setup.py
   src
   tests
   tox.ini
From "/notebooks/notebooks" to "/notebooks/notebooks/../src"
   wapor
   WaPOR.egg-info


## 1. First import the module
Init module with API input.

In [2]:
# from wapor.WaporAPI import __wapor_API_class
# API = __wapor_API_class()
# API.Token=input('Insert WAPOR API Token: ')

import wapor

Insert WAPOR API Token: ae34c8743c4dc4b3c32d26501fcef18b0cc47464baaa87cceb1b10d5ee1096ba03ab36196d29fe07
WaPOR: Loading sign-in...
WaPOR:   _query_accessToken
https://io.apps.fao.org/gismgr/api/v1/iam/sign-in/


In [3]:
APIToken = wapor.API.token['API']
AccToken = wapor.API.token['Access']
RefToken = wapor.API.token['Refresh']

print('\nAPI     Token:', APIToken)
print('\nAccess  Token:', AccToken)
print('\nRefresh Token:', RefToken)


API     Token: ae34c8743c4dc4b3c32d26501fcef18b0cc47464baaa87cceb1b10d5ee1096ba03ab36196d29fe07

Access  Token: eyJhbGciOiJSUzI1NiIsImtpZCI6ImEwYjQwY2NjYmQ0OWQxNmVkMjg2MGRiNzIyNmQ3NDZiNmZhZmRmYzAiLCJ0eXAiOiJKV1QifQ.eyJuYW1lIjoiUXVhbiBQYW4iLCJwaWN0dXJlIjoiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EtL0FBdUU3bUR2LWQ4NEF5OVE4UmVaY19aWHlRakVjSkxHc2NpMGdURVFQaTFtN2ciLCJpc3MiOiJodHRwczovL3NlY3VyZXRva2VuLmdvb2dsZS5jb20vdW5mYW8tYXBwcy1kZXYiLCJhdWQiOiJ1bmZhby1hcHBzLWRldiIsImF1dGhfdGltZSI6MTU3MTc1NDQzMywidXNlcl9pZCI6IkpHSXp0WGN6RndUdm9DaUR2bmNXdFoyeFBzaTEiLCJzdWIiOiJKR0l6dFhjekZ3VHZvQ2lEdm5jV3RaMnhQc2kxIiwiaWF0IjoxNTcxNzU0NDMzLCJleHAiOjE1NzE3NTgwMzMsImVtYWlsIjoicXVhbnBhbjMwMkBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZmlyZWJhc2UiOnsiaWRlbnRpdGllcyI6eyJnb29nbGUuY29tIjpbIjExMDA1MTgyOTI3MDIxODc5MzAwOCJdLCJlbWFpbCI6WyJxdWFucGFuMzAyQGdtYWlsLmNvbSJdfSwic2lnbl9pbl9wcm92aWRlciI6ImN1c3RvbSJ9fQ.L7nYnYbCQO7a14msXs0xJMce-S5Ec3O8ufXOHuPUxbFJ0az_aiG36Uu7oiwei2HEsVky4RxHrWhuORwS-hPzucd9Sdr6tM3UZRRdIndwI

Check Workspaces list

In [4]:
workspaces = wapor.API.getWorkspaces()

workspaces

WaPOR: Loading workspace...
WaPOR:   _query_workspaces
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/?overview=false&paged=false&sort=code


[{'code': 'RDMS',
  'caption': 'Regional Drought Monitoring System',
  'description': 'Regional Drought Monitoring System for Middle East and North Africa',
  'links': [{'rel': 'self',
    'href': 'https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/RDMS'}]},
 {'code': 'RICCAR',
  'caption': 'Regional Arab Climate Change Assessment Report',
  'description': 'The Regional Initiative for the Assessment of Climate Change Impacts on Water Resources and Socio-Economic Vulnerability in the Arab Region (RICCAR) is a joint initiative of the United Nations and the League of Arab States launched in 2010.',
  'additionalInfo': {'tagContexts': {'LEVELS_LIST': [{'name': 'L1',
      'label': 'CLIMATE'},
     {'name': 'L2', 'label': 'HYDROLOGY'},
     {'name': 'L3', 'label': 'VULNERABILITY'}],
    'CATEGORIES_LIST': [{'name': 'CDD',
      'label': 'Maximum length of dry spell',
      'level': 'L1'},
     {'name': 'CWD', 'label': 'Maximum length of wet spell', 'level': 'L1'},
     {'name': 'PR', 

## 2. Get Catalog

In [5]:
wapor.API.version = 2
catalog = wapor.API.getCatalog()

# APIVersion = 1
# catalog = wapor.API.getCatalog(level=APIVersion, cubeInfo=True)

catalog

WaPOR: Loading catalog...
WaPOR:   _query_catalog
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L1_GBWP_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L1_GBWP_A/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L1_NBWP_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L1_NBWP_A/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L1_AETI_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L1_AETI_A/di

WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_AETI_D/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_T_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_T_A/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_E_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_E_A/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_I_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_I_A/dimensions?overvie

WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_BKA_T_A/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_KOG_T_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_KOG_T_A/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_ODN_T_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_ODN_T_A/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_ZAN_T_A/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_Z

WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_ZAN_I_D/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_AWA_NPP_D/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_AWA_NPP_D/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_BKA_NPP_D/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_BKA_NPP_D/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L3_KOG_NPP_D/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/

WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_CTY_NPP_D/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_CTY_I_D/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_CTY_I_D/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_BAS_TBP_S/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_BAS_TBP_S/dimensions?overview=false&paged=false
WaPOR:   _query_cubeMeasures
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L2_CTY_PHE_S/measures?overview=false&paged=false
WaPOR:   _query_cubeDimensions
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cu

,code,caption,description,additionalInfo,tags,workspaceCode,dataType,index,operation,hidden,links,measure,dimension
0,L1_GBWP_A,Gross Biomass Water Productivity,The annual Gross Biomass Water Productivity ex...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 1, 'context': 'LEVELS...",WAPOR_2,RASTER,101,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
1,L1_NBWP_A,Net Biomass Water Productivity,The annual Net Biomass Water Productivity expr...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 2, 'context': 'LEVELS...",WAPOR_2,RASTER,102,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
2,L1_AETI_A,Actual EvapoTranspiration and Interception (An...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 3, 'context': 'LEVELS...",WAPOR_2,RASTER,103,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
3,L1_AETI_M,Actual EvapoTranspiration and Interception (Mo...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 4, 'context': 'LEVELS...",WAPOR_2,RASTER,104,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'MONTH', 'caption': 'Month', 'worksp..."
4,L1_AETI_D,Actual EvapoTranspiration and Interception (De...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 5, 'context': 'LEVELS...",WAPOR_2,RASTER,105,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,L2_BAS_PHE_S,"Phenology (Seasonal, clipped by basin)","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4151,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'PHE', 'caption': 'Phenology', 'descr...","[{'code': 'YEAR', 'caption': 'Year', 'workspac..."
119,L2_BAS_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4161,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
120,L2_CTY_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4162,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
121,L2_BAS_QUAL_LST_D,"Quality Land Surface Temperature (Dekadal, cli...","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4171,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."


## 3. Get information of a cube

In [6]:
cube_code='L1_PCP_M'
cube_info=wapor.API.getCubeInfo(cube_code)

cube_info

WaPOR: Loading "L1_PCP_M" info...


{'code': 'L1_PCP_M',
 'caption': 'Precipitation (Monthly)',
 'description': 'Precipitation data is delivered on a daily basis. The source of this dataset is CHIRPS (Climate Hazards Group InfraRed Precipitation with Station) quasi-global rainfall dataset, starting from 1981 up to near present. The value of each pixel represents the total of daily precipitation in the month expressed in mm (1mm=1l/m² or 1mm=10m³/ha).',
 'additionalInfo': {'format': 'Raster Dataset',
  'unit': 'mm (1mm=1l/m² or 1mm=10m³/ha)',
  'dataType': 'Int16 (16bit Integer)',
  'conversionFactor': 'the pixel value in the downloaded data must be multiplied by 0.1',
  'noDataValue': -9999,
  'spatialResolution': 'Approximately 5km (0.05 degree)',
  'spatialExtent': 'Africa and Near East',
  'spatialReferenceSystem': 'EPSG:4326 - WGS84 - Geographic Coordinate System (lat/long)',
  'temporalResolution': 'from January 2009 to present',
  'temporalExtent': 'Monthly',
  'nearRealTime': 'New dekadal data layers are released 

Some useful information about dataset can be found in cube_info

In [7]:
print('Description: {0} \n\nmultiplier: {1} \n\nunit: {2} \n'.format(
    cube_info['description'],
    cube_info['measure']['multiplier'],
    cube_info['measure']['unit']))

Description: Precipitation data is delivered on a daily basis. The source of this dataset is CHIRPS (Climate Hazards Group InfraRed Precipitation with Station) quasi-global rainfall dataset, starting from 1981 up to near present. The value of each pixel represents the total of daily precipitation in the month expressed in mm (1mm=1l/m² or 1mm=10m³/ha). 

multiplier: 0.1 

unit: mm 



## 4. Get list of locations

In [8]:
df_locations=wapor.API.getLocations()

df_locations

WaPOR: Loading locations...


## 5. Get list of available data

In [9]:
df_avail=wapor.API.getAvailData(cube_code,
                                time_range='2009-01-01,2012-12-31')

df_avail

WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" data...
WaPOR:   _query_dimensionsMembers
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/cubes/L1_PCP_M/dimensions/MONTH/members?overview=false&paged=false
WaPOR:   _query_availData
https://io.apps.fao.org/gismgr/api/v1/query/


,MONTH,raster_id,bbox,time_code
0,2009-01,L1_PCP_0901M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-01-01,2009-02-01)"
1,2009-02,L1_PCP_0902M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-02-01,2009-03-01)"
2,2009-03,L1_PCP_0903M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-03-01,2009-04-01)"
3,2009-04,L1_PCP_0904M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-04-01,2009-05-01)"
4,2009-05,L1_PCP_0905M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-05-01,2009-06-01)"
5,2009-06,L1_PCP_0906M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-06-01,2009-07-01)"
6,2009-07,L1_PCP_0907M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-07-01,2009-08-01)"
7,2009-08,L1_PCP_0908M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-08-01,2009-09-01)"
8,2009-09,L1_PCP_0909M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-09-01,2009-10-01)"
9,2009-10,L1_PCP_0910M,"[{'srid': 'EPSG:4326', 'value': [-30.05, -40.0...","[2009-10-01,2009-11-01)"


## 6. Get download url of a raster
To get download url, wapor API Token is needed

In [10]:
# # Input API token
# # or 
# # Init instance with API token, already
# APIToken=input('Enter your API Token: ')
# wapor.API.Token=APIToken

wapor.API.getRasterUrl(cube_code,
                       'L1_PCP_0901M',
                       APIToken)

WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" url...
WaPOR:   _query_rasterUrl
https://io.apps.fao.org/gismgr/api/v1/download/WAPOR_2


{'url': 'https://io.apps.fao.org/gismgr/download/3a4c04eb-ecd9-4d27-bb08-dfb8d272f148/L1_PCP_0901M.tif',
 'expiry_datetime': datetime.datetime(2019, 10, 22, 17, 27, 53, 495021)}

## 7. Get download url of a cropped raster
To request a cropped raster, raster_id and time_code of the raster is needed

In [18]:
bbox= [37.95883206252312, 7.89534, 43.32093, 12.3873979377346]
rasterId='L1_PCP_0901M'
time_code='[2009-01-01,2009-02-01)'
download_url=wapor.API.getCropRasterURL(bbox,
                                        cube_code,
                                        time_code,
                                        rasterId,
                                        APIToken)

download_url

WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/7a7d82de-d860-4093-bbd1-2e9938d678e1
  0 WAITING
  1 WAITING
  2 RUNNING
  3 RUNNING
  4 RUNNING
  5 COMPLETED


'https://io.apps.fao.org/gismgr/download/31e11237-9fe8-44bc-93c0-08e01d239260/L1_PCP_0901M.tif'

## 8. Get Area time-series
A shapefile can be used to define the Region of Interest

### 8.1 Check Shapefile folder

In [12]:
Dir_Shapefile = os.path.join(p_tests, 'data', 'Shapefile')
chk_files=glob.glob(os.path.join(Dir_Shapefile, '*.shp'))

for file in chk_files:
    print(file)

/notebooks/notebooks/../tests/data/Shapefile/Awash_HydroSHED.shp


### 8.2 Get shapefile area timeseries data

In [13]:
shp_fh=os.path.join(Dir_Shapefile, "Awash_HydroSHED.shp")
ts_area=wapor.API.getAreaTimeseries(shp_fh,
                                    'L1_PCP_M',
                                    APIToken,
                                    time_range="2009-01-01,2018-12-31")

ts_area

WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" area timeseries...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "AREA STATS"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/85f3b1ec-e5bc-4169-b7b4-12cd49dec7f0
  0 WAITING
  1 WAITING
  2 RUNNING
  3 RUNNING
  4 COMPLETED


,month,avg,min,max,range
0,2009-01-31,25.645787,0.0,66.3,66.3
1,2009-02-28,7.826904,0.0,25.4,25.4
2,2009-03-31,33.049873,4.4,105.2,100.8
3,2009-04-30,36.046142,0.0,155.1,155.1
4,2009-05-31,20.727437,0.0,102.4,102.4
...,...,...,...,...,...
115,2018-08-31,141.077563,39.4,352.8,313.4
116,2018-09-30,48.423934,4.3,175.2,170.9
117,2018-10-31,30.099797,0.0,148.8,148.8
118,2018-11-30,22.768579,0.0,73.9,73.9


## 9. Get Point time-series

In [14]:
Point=[37.95883206252312, 7.89534] #longitude and latitude
ts_point=wapor.API.getPixelTimeseries(Point,
                                      'L1_PCP_M',
                                      time_range="2009-01-01,2018-12-31")

ts_point

WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" point timeseries...
https://io.apps.fao.org/gismgr/api/v1/query/


,month,value
0,2009-01-31,60.0
1,2009-02-28,20.5
2,2009-03-31,84.0
3,2009-04-30,92.3
4,2009-05-31,83.8
...,...,...
115,2018-08-31,223.1
116,2018-09-30,148.8
117,2018-10-31,51.4
118,2018-11-30,37.9


## 10. Bulk-downloading

### 10.1 Check Download folder

In [15]:
import requests
Dir_Download = os.path.join(p_tests, 'data', 'Download')
chk_files=glob.glob(os.path.join(Dir_Download, '*.tif'))

for file in chk_files:
    print(file)

### 10.2 Start to download

In [16]:
for index,row in df_avail.iterrows():   
    download_url=wapor.API.getCropRasterURL(bbox,
                                            cube_code,
                                            row['time_code'],
                                            row['raster_id'],
                                            APIToken)
    
    download_file=os.path.join(Dir_Download,
                               '{0}.tif'.format(row['raster_id']))
    
    resp=requests.get(download_url) 
    open(download_file, 'wb').write(resp.content)
    print(index, download_file)

WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/b994b8b5-ac7d-40a3-8d73-34e37c126557
  0 WAITING
  1 WAITING
  2 RUNNING
  3 RUNNING
  4 COMPLETED
0 /notebooks/notebooks/../tests/data/Download/L1_PCP_0901M.tif
WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/6620b092-4fae-45fe-9d0f-0fcf361a4578
  0 WAITING
  1 WAITING
  2 RUNNING
  3 RUNNING
  4 COMPLETED
1 /notebooks/notebooks/../tests/data/Download/L1_PCP_0902M.tif
WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/

  1 WAITING
  2 RUNNING
  3 RUNNING
  4 COMPLETED
18 /notebooks/notebooks/../tests/data/Download/L1_PCP_1007M.tif
WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/f98ee193-14d2-4006-bea8-e71b47ebbd8c
  0 WAITING
  1 WAITING
  2 RUNNING
  3 RUNNING
  4 RUNNING
  5 COMPLETED
19 /notebooks/notebooks/../tests/data/Download/L1_PCP_1008M.tif
WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/b1801902-b814-4e62-9be5-62997c9b0519
  0 WAITING
  1 RUNNING
  2 RUNNING
  3 RUNNING
  4 COMPLETED
20 /notebooks/notebooks/../tests/data/Download/L1_PC

WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/77b38319-6db6-4aa7-97f8-6b9f37261779
  0 WAITING
  1 WAITING
  2 RUNNING
  3 RUNNING
  4 RUNNING
  5 COMPLETED
37 /notebooks/notebooks/../tests/data/Download/L1_PCP_1202M.tif
WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismgr/api/v1/catalog/workspaces/WAPOR_2/jobs/4779438f-1fc8-4063-b96f-37d1fac5405d
  0 WAITING
  1 WAITING
  2 RUNNING
  3 RUNNING
  4 COMPLETED
38 /notebooks/notebooks/../tests/data/Download/L1_PCP_1203M.tif
WaPOR: Checking token...
WaPOR: Loading "L1_PCP_M" info...
WaPOR: Loading "L1_PCP_M" url...
https://io.apps.fao.org/gismgr/api/v1/query/
WaPOR: Downloading "L1_PCP_M" "CROP RASTER"...
WaPOR:   _query_jobOutput
https://io.apps.fao.org/gismg

### 10.3 Check downloaded files

In [17]:
chk_files=glob.glob(os.path.join(Dir_Download, '*.tif'))
for file in chk_files:
    print(file)

/notebooks/notebooks/../tests/data/Download/L1_PCP_0901M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0902M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0903M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0904M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0905M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0906M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0907M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0908M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0909M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0910M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0911M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_0912M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_1001M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_1002M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_1003M.tif
/notebooks/notebooks/../tests/data/Download/L1_PCP_1004M.tif
/notebooks/notebooks/../